In [9]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine, text
from urllib.parse import quote_plus
import numpy as np
# === Paramètres de connexion ===
user = "postgres"
password = quote_plus("attijaribank123456789")
host = "localhost"
port = "5432"
db_main = "postgres"
db_target = "ashref_dw"

# === Étape 1 : Création de la base si elle n'existe pas ===
conn_main = psycopg2.connect(dbname=db_main, user=user, password="attijaribank123456789", host=host, port=port)
conn_main.autocommit = True
cursor = conn_main.cursor()
cursor.execute(f"SELECT 1 FROM pg_database WHERE datname = '{db_target}'")
if not cursor.fetchone():
    cursor.execute(f"CREATE DATABASE {db_target};")
    print(f"✅ Base de données '{db_target}' créée.")
else:
    print(f"ℹ️ La base '{db_target}' existe déjà.")
cursor.close()
conn_main.close()

✅ Base de données 'ashref_dw' créée.


In [13]:
# === Étape 2 : Connexion SQLAlchemy à la base ===
engine = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{db_target}")

# === Étape 3 : Définition des tables (ordre logique) ===

schema_sql_1 = """
CREATE TABLE dim_date (
    id_date VARCHAR PRIMARY KEY,
    date_calendrier DATE,
    jour INT,
    mois INT,
    annee INT,
    trimestre INT,
    semestre INT,
    jour_semaine INT,
    numero_semaine INT,
    jour_annee INT,
    nom_jour VARCHAR(20),
    nom_mois VARCHAR(20),
    est_weekend BOOLEAN,
    est_debut_mois BOOLEAN,
    est_fin_mois BOOLEAN,
    nom_trimestre VARCHAR(20),
    nom_semestre VARCHAR(20),
    mois_annee VARCHAR(30)
);

CREATE TABLE dim_clients (
    id_client VARCHAR PRIMARY KEY,
    type_client VARCHAR(50),
    sexe VARCHAR(50),
    age INT,
    tranche_age VARCHAR(20),
    date_naissance DATE,
    salaire NUMERIC(10,2),
    tranche_salaire VARCHAR(20),
    emploi VARCHAR(100),
    statut_marital VARCHAR(50),
    niveau_education VARCHAR(50),
    ville VARCHAR(100)
);

CREATE TABLE dim_type_operation (
    id_type_operation INT PRIMARY KEY,
    type_operation VARCHAR(100),
    sens_operation VARCHAR(20),
    canal_operation VARCHAR(50),
    nature_operation VARCHAR(50)
);
"""

schema_sql_2 = """
CREATE TABLE dim_comptes (
    id_compte VARCHAR PRIMARY KEY,
    id_client VARCHAR,
    type_compte VARCHAR(50),
    type_carte VARCHAR(50),
    date_ouverture DATE,
    solde_initial NUMERIC(12,2),
    tranche_solde VARCHAR(20),
    etat_compte VARCHAR(20),
    eligible_chequier BOOLEAN,
    deja_cheque BOOLEAN,
    demande_cheque BOOLEAN,
    agence VARCHAR(100),
    anciennete_compte INT,
    tranche_anciennete VARCHAR(20),
    FOREIGN KEY (id_client) REFERENCES dim_clients(id_client)
);

CREATE TABLE fact_operations (
    id_operation VARCHAR PRIMARY KEY,
    id_compte VARCHAR,
    id_date VARCHAR,
    id_type_operation INT,
    montant_total NUMERIC(12,2),
    montant_absolu NUMERIC(12,2),
    lieu_operation VARCHAR(100),
    montant_par_cheque NUMERIC(10,2),
    nombre_cheques INT,
    sens_operation VARCHAR(20),
    solde_avant NUMERIC(12,2),
    solde_apres NUMERIC(12,2),
    est_gros_montant BOOLEAN,
    FOREIGN KEY (id_compte) REFERENCES dim_comptes(id_compte),
    FOREIGN KEY (id_date) REFERENCES dim_date(id_date),
    FOREIGN KEY (id_type_operation) REFERENCES dim_type_operation(id_type_operation)
);"""

In [15]:
# === Étape 5 : Chargement des fichiers CSV ===
print("\n📥 Chargement des fichiers CSV...")

df_clients = pd.read_csv("C:/Users/MSI/Desktop/Attijari_bank/stage/dim_clients.csv", dtype={"id_client": str})
df_comptes = pd.read_csv("C:/Users/MSI/Desktop/Attijari_bank/stage/dim_comptes.csv", dtype={"id_client": str, "id_compte": str})
df_date = pd.read_csv("C:/Users/MSI/Desktop/Attijari_bank/stage/dim_date.csv", dtype={"id_date": str})
df_type_operation = pd.read_csv("C:/Users/MSI/Desktop/Attijari_bank/stage/dim_type_operation.csv", dtype={"id_type_operation": int})
df_fact = pd.read_csv("C:/Users/MSI/Desktop/Attijari_bank/stage/fact_operations.csv", dtype={
    "id_operation": str, "id_compte": str, "id_date": str, "id_type_operation": int
})

print(f"   - dim_clients: {len(df_clients):,} lignes")
print(f"   - dim_comptes: {len(df_comptes):,} lignes")
print(f"   - dim_date: {len(df_date):,} lignes")
print(f"   - dim_type_operation: {len(df_type_operation):,} lignes")
print(f"   - fact_operations: {len(df_fact):,} lignes")


📥 Chargement des fichiers CSV...
   - dim_clients: 180,000 lignes
   - dim_comptes: 359,723 lignes
   - dim_date: 1,096 lignes
   - dim_type_operation: 33 lignes
   - fact_operations: 67,411,391 lignes


In [18]:
# === Étape 6 : Export vers PostgreSQL avec vérification ===
try:
    print("\n🚀 Exportation vers PostgreSQL...")

    df_date.to_sql("dim_date", engine, if_exists="append", index=False)
    print("   ✅ dim_date exportée")

    df_clients.to_sql("dim_clients", engine, if_exists="append", index=False)
    print("   ✅ dim_clients exportée")

    df_type_operation.to_sql("dim_type_operation", engine, if_exists="append", index=False)
    print("   ✅ dim_type_operation exportée")

    df_comptes.to_sql("dim_comptes", engine, if_exists="append", index=False)
    print("   ✅ dim_comptes exportée")

    # Export avec messages pour fact_operations (avec indentation correcte)
    print("\n🚀 Exportation de fact_operations par morceaux de 10 000 lignes...")

    num_chunks = len(df_fact) // 10000 + 1  # Nombre total de morceaux

    for i, chunk in enumerate(np.array_split(df_fact, num_chunks)):
        chunk.to_sql("fact_operations", engine, if_exists="append", index=False)
        print(f"   ✅ Chunk {i+1}/{num_chunks} exporté ({len(chunk)} lignes)")

    print("✅ Export complet de fact_operations terminé.")

    # Vérification des lignes exportées
    with engine.connect() as conn:
        print("\n📊 Vérification des lignes insérées :")
        for table in ["dim_date", "dim_clients", "dim_type_operation", "dim_comptes", "fact_operations"]:
            result = conn.execute(text(f"SELECT COUNT(*) FROM {table}"))
            count = result.fetchone()[0]
            print(f"   - {table}: {count:,} lignes")

    print("\n✅ Export terminé sans perte d'information.")

except Exception as e:
    print(f"❌ Erreur pendant l'export : {e}")

finally:
    engine.dispose()
    print("\n🔌 Connexion PostgreSQL fermée.")
    input("Appuyez sur Entrée pour quitter...")



🚀 Exportation vers PostgreSQL...
   ✅ dim_date exportée
   ✅ dim_clients exportée
   ✅ dim_type_operation exportée
   ✅ dim_comptes exportée

🚀 Exportation de fact_operations par morceaux de 10 000 lignes...


C:\Users\MSI\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


   ✅ Chunk 1/6742 exporté (9999 lignes)
   ✅ Chunk 2/6742 exporté (9999 lignes)
   ✅ Chunk 3/6742 exporté (9999 lignes)
   ✅ Chunk 4/6742 exporté (9999 lignes)
   ✅ Chunk 5/6742 exporté (9999 lignes)
   ✅ Chunk 6/6742 exporté (9999 lignes)
   ✅ Chunk 7/6742 exporté (9999 lignes)
   ✅ Chunk 8/6742 exporté (9999 lignes)
   ✅ Chunk 9/6742 exporté (9999 lignes)
   ✅ Chunk 10/6742 exporté (9999 lignes)
   ✅ Chunk 11/6742 exporté (9999 lignes)
   ✅ Chunk 12/6742 exporté (9999 lignes)
   ✅ Chunk 13/6742 exporté (9999 lignes)
   ✅ Chunk 14/6742 exporté (9999 lignes)
   ✅ Chunk 15/6742 exporté (9999 lignes)
   ✅ Chunk 16/6742 exporté (9999 lignes)
   ✅ Chunk 17/6742 exporté (9999 lignes)
   ✅ Chunk 18/6742 exporté (9999 lignes)
   ✅ Chunk 19/6742 exporté (9999 lignes)
   ✅ Chunk 20/6742 exporté (9999 lignes)
   ✅ Chunk 21/6742 exporté (9999 lignes)
   ✅ Chunk 22/6742 exporté (9999 lignes)
   ✅ Chunk 23/6742 exporté (9999 lignes)
   ✅ Chunk 24/6742 exporté (9999 lignes)
   ✅ Chunk 25/6742 export

Appuyez sur Entrée pour quitter... 
